The main solving big ridge regression problems

In [ ]:
workspace()

In [ ]:
using Plots

In [ ]:
using StatsBase
using JLD  
include("../src/BigRidge.jl")

In [67]:
#Basic parameters
## Basic parameters
maxiter=10^8;
max_time = 10;
max_epocs = 100;
printiters = true;
exacterror =true; 
tol = 10.0^(-5.0);
restol = 10.0^(-5.0);
skip_error_calculation = 25;   # number of iterations where error is not calculated (to save time!) ceil((m)^(1/2))

In [68]:
dataset = "a9a"
prob = defineProb(dataset);

../data/a9a.jld


In [69]:
options = MyOptions(0,"uni",tol,restol,maxiter,skip_error_calculation,max_time,printiters,exacterror) 
OUTPUTS = [];

In [70]:
# Pkg.add("Hadamard")
# using Hadamard 

m = 64;
idx = sample(1:m,10,replace=true);
A = rand(m,m)
mat = hada(A,idx);


H = hadamard(m);
HA =H*A;
error = norm(HA[idx,:] -mat)

7.14194331727446e-15

In [71]:
# Testing Hadamard Sketch
method_name = "Hadamard";# ROS
options.sketchsize =prob.n; # Int(floor(log2(prob.n)));

output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];

Hadamard
-------------------
It   | Error% | Residual |  Time   
-------------------
 25  | 26854039412004656.00  |  22991407609931.97  | 1.7335 


In [72]:
# Testing CG 
options.max_time = 2; options.restol = 10.0^(-8); 
method_name = "CG";# gradient descent
output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];

CG
-------------------
It   | Error% | Residual |  Time   
-------------------
 25  | 35.08  |  0.06  | 0.0250 
 50  | 10.02  |  0.02  | 0.0255 
 75  | 4.13  |  0.01  | 0.0258 
100  | 3.15  |  0.00  | 0.0262 
125  | 2.37  |  0.00  | 0.0266 
150  | 0.30  |  0.00  | 0.0270 
175  | 0.01  |  0.00  | 0.0274 
200  | 0.00  |  0.00  | 0.0278 


In [73]:
# Testing grad descent
options.max_time = 2; options.restol = 10.0^(-8); options.maxiter=1000;
method_name = "grad";# gradient descent
output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];

grad
-------------------
It   | Error% | Residual |  Time   
-------------------
 25  | 73.17  |  1.45  | 0.0249 
 50  | 67.96  |  1.54  | 0.0291 
 75  | 65.17  |  0.44  | 0.0313 
100  | 63.12  |  0.71  | 0.0331 
125  | 61.61  |  0.24  | 0.0347 
150  | 60.29  |  0.43  | 0.0364 
175  | 59.19  |  0.16  | 0.0381 
200  | 58.15  |  0.31  | 0.0398 
225  | 57.25  |  0.12  | 0.0621 
250  | 56.36  |  0.24  | 0.0724 
275  | 55.56  |  0.10  | 0.0738 
300  | 54.77  |  0.20  | 0.0765 
325  | 54.05  |  0.08  | 0.0780 
350  | 53.33  |  0.18  | 0.0794 
375  | 52.67  |  0.08  | 0.0825 
400  | 52.00  |  0.16  | 0.0840 
425  | 51.39  |  0.07  | 0.0854 
450  | 50.76  |  0.15  | 0.0868 
475  | 50.19  |  0.06  | 0.0881 
500  | 49.61  |  0.14  | 0.0900 
525  | 49.07  |  0.06  | 0.0913 
550  | 48.52  |  0.13  | 0.0950 
575  | 48.01  |  0.06  | 0.1022 
600  | 47.49  |  0.12  | 0.1085 
625  | 47.01  |  0.05  | 0.1097 
650  | 46.51  |  0.11  | 0.1189 
675  | 46.06  |  0.05  | 0.1200 
700  | 45.59  |  0.11  | 0.1

In [81]:
# Testing CD 
options.max_time = 10; options.restol = 10.0^(-8); options.maxiter=10000;
method_name = "CD";# gradient descent
options.sketchsize = 11#floor((prob.n)^(1/2));
output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];

CD
-------------------
It   | Error% | Residual |  Time   
-------------------
 25  | 125.40  |  3.06  | 0.0010 
 50  | 122.14  |  1.19  | 0.0019 
 75  | 117.73  |  0.41  | 0.0028 
100  | 113.81  |  0.22  | 0.0036 
125  | 111.34  |  0.30  | 0.0045 
150  | 109.36  |  0.20  | 0.0053 
175  | 105.00  |  0.18  | 0.0061 
200  | 101.98  |  0.23  | 0.0070 
225  | 100.53  |  0.16  | 0.0079 
250  | 99.47  |  0.15  | 0.0087 
275  | 98.22  |  0.23  | 0.0095 
300  | 96.61  |  0.14  | 0.0104 
325  | 95.79  |  0.14  | 0.0113 
350  | 94.98  |  0.11  | 0.0121 
375  | 94.65  |  0.13  | 0.0129 
400  | 94.95  |  0.08  | 0.0137 
425  | 94.64  |  0.09  | 0.0146 
450  | 94.68  |  0.12  | 0.0154 
475  | 94.73  |  0.10  | 0.0163 
500  | 99.00  |  0.07  | 0.0171 
525  | 100.87  |  0.03  | 0.0179 
550  | 100.83  |  0.02  | 0.0186 
575  | 100.73  |  0.02  | 0.0194 
600  | 100.72  |  0.02  | 0.0213 
625  | 100.79  |  0.01  | 0.0221 
650  | 100.78  |  0.01  | 0.0229 
675  | 100.73  |  0.01  | 0.0237 
700  | 100.69 

In [ ]:
savename = replace(prob.name, r"[\/]", "-");
save("../data/$(savename)-OUTPUTS.jld", "OUTPUTS", OUTPUTS)

In [ ]:
#using GR
using Plots
pgfplots()
plot_outputs_Plots(OUTPUTS,savename)

In [ ]:
# Loading saved problem data
probname = "w8a"#"gisette_scale-ridge";  
test = load("../data/$(probname)-OUTPUTS.jld")
OUTPUTS = test["OUTPUTS"];
# mask = [1,3,5]
# OUTPUTS[mask]